In [ ]:
import sys
sys.path.append("..") # for sibling import

import compyute as cp

# Example 3.1

### Deep neural network using sequential model

The goal of this model is to classify iris species based on numerical features.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/uciml/iris and place it into the *data* directory.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd

data_orig = pd.read_csv('../data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

The labels are categorical values. To be used in the model, all data needs to be numerical.

In [ ]:
data_enc = data.copy()
data_enc["Species"] = data_enc["Species"].astype("category").cat.codes
data_enc.head()

Convert dataframe to a tensor.

In [ ]:
data_tensor = cp.Tensor(data_enc.to_numpy())
data_tensor.shape

Next the data is split into a training, validation and a testing dataset using the `split_train_test_val_data()` to evaluate the model later on. Before splitting the data is also shuffled, since sometimes raw data is sorted in some way.

In [ ]:
from compyute.preprocessing import split_train_val_test

train, val, test = split_train_val_test(data_tensor, ratio_val=0.25, ratio_test=0.25)
train[:5]

Features and labels are now seperated.

In [ ]:
X_train, y_train = train[:, :-1], train[:, -1].int()
X_val, y_val = val[:, :-1], val[:, -1].int()
X_test, y_test = test[:, :-1], test[:, -1].int()

In [ ]:
y_train[:10]

Neural networks tend to run into problems if values are very high. Therefore it is common to normalize the data. This can be done using the `normalize()` function, which applies min-max feature scaling to a tensor.<br><br>
$ X'=a+\frac{(X-X_{min})\cdot(b-a)}{X_{max}-X_{min}} $<br><br>, where<br><br>$ a $ ... lower bound<br>$ b $ ... upper bound

In [ ]:
from compyute.preprocessing import normalize

X_train = normalize(X_train, axis=0, l_bound=-1)
X_val = normalize(X_val, axis=0, l_bound=-1)
X_test = normalize(X_test, axis=0, l_bound=-1)
X_train[:5]

In [ ]:
print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

### Step 2: Build the neural network structure
Here the individual layers of the neural network models are defined.

In [ ]:
from compyute import nn

model = nn.Sequential([
    nn.Linear(4, 16),
    nn.ReLU(),
    nn.Batchnorm1d(16),
    nn.Linear(16, 3),
])

The network is compiled to finalize the model. Besides the SGD optimizer, the framework also provides other algorithms like Adam. There are also multiple loss functions to choose from. Since this example explores a classification problem, the cross entropy loss function is used.

In [ ]:
model.summary(input_shape=(4,))

### Step 3: Train the model

In [ ]:
from compyute.nn.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer="sgd",
    loss="cross_entropy",
    metric="accuracy",
    verbose=1
)

In [ ]:
trainer.train(X_train, y_train, epochs=300, val_data=(X_val, y_val))

In [ ]:
# ! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plot_history(t1, t2):
    trace1 = trainer.state[t1]
    trace2 = trainer.state[t2]
    plt.figure(figsize=(10, 3))
    plt.plot(cp.arange(start=1, stop=len(trace1) + 1).to_numpy(), trace1, linewidth=1)
    plt.plot(cp.arange(start=1, stop=len(trace2) + 1).to_numpy(), trace2, linewidth=1)
    plt.legend([t1, t2])
    plt.grid(color="gray", linestyle="--", linewidth=0.5)

In [ ]:
plot_history("loss", "val_loss")

In [ ]:
plot_history("accuracy", "val_accuracy")

### Step 4: Evaluate the model
Using the defined metric, the model's performance can be evaluated using testing/validation data.

In [ ]:
loss, accuracy = trainer.evaluate_model(X_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')